In [1]:
!activate PythonGPU
import numpy as np
from scipy.stats import skewnorm, skew
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, accuracy_score

def simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew):
    #The multivariate skew normal number generator
    def rng(mu, sigma, skew, n=1):
        k = len(mu)
        if not (k == len(sigma) and k ==len(skew)): 
            raise Exception("Mu, Sigma and Skew should be same length")

        data = np.zeros((int(n),k))

        for i in range(k):
            data[:,i] = skewnorm.rvs(skew[i], loc=mu[i], scale=sigma[i], size=int(n)) 

        return data
    
    if(np.sum(classes) != 1):
        raise Exception("Classes dont sum up to 1")
        
    n_classes = len(classes)    
    sigma = (max_sigma * np.random.rand(1,n_vars))[0]
    skew = ((2 * max_skew  * np.random.rand(1, n_vars)) - max_skew)[0]
    mu = (2 *  max_mu * np.random.rand(n_classes, n_vars)) - max_mu
    
    n_obs_class = np.round(np.dot(classes,n))
    
    data = np.zeros((int(np.sum(n_obs_class)),n_vars+1))
    for i in range(n_classes):
        #calculate indexes
        start = int(np.sum(n_obs_class[0:i]))
        end = int(np.sum(n_obs_class[0:i+1]))
        
        #set the data
        data[start:end,0] = i
        data[start:end,1:] = rng(mu[i,:], sigma, skew, n_obs_class[i])
        
    X = data[:,1:]
    y = data[:,0]
        
    X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42,
    stratify=y)
    
    return X_train, X_test, y_train, y_test

In [2]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def classify_lda(X_train, X_test, y_train, y_test, priors, plot=False):
    lda = LinearDiscriminantAnalysis(priors=priors)
    X_lda = lda.fit_transform(X_train, y_train)

    predictions = lda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("LDA Test accuracy "+ str(accuracy))
    print(predictions)

    if plot:    
        plt.xlabel('LD1')
        plt.ylabel('LD2')
        plt.scatter(
            X_lda[:,0],
            X_lda[:,1],
            c=y_train,
            cmap='Accent',
        )
        
    return {"method": "LDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": lda}

In [3]:
#Quadratic
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify_qda(X_train, X_test, y_train, y_test, priors):
    qda = QuadraticDiscriminantAnalysis(priors=priors)
    X_qda = qda.fit(X_train, y_train)



    predictions = qda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("QDA Test accuracy "+ str(accuracy))

    return {"method": "QDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": qda}

In [4]:
from sklearn.linear_model import LogisticRegression

def classify_logit(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0, solver='lbfgs',
                             multi_class='multinomial').fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Logistic Test accuracy "+ str(accuracy))
    
    return {"method": "Logit", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": clf}

In [5]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

def classify_knn(X_train, X_test, y_train, y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(X_train, y_train)

    predictions = knn.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("KNN-"+str(n_neighbors)+" Test accuracy "+ str(accuracy))
    
    return {"method": "KNN-"+str(n_neighbors), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": knn}

In [6]:
#Naive bayes
from sklearn.naive_bayes import GaussianNB

def classify_naivebayes(X_train, X_test, y_train, y_test, priors):
    NB = GaussianNB(priors)
    NB.fit(X_train, y_train)
    
    predictions = NB.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("Naive Bayes Test accuracy "+ str(accuracy))
    
    return {"method": "Naive Bayes", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": NB}

In [7]:
#SVM
from sklearn.svm import LinearSVC

def classify_svm(X_train, X_test, y_train, y_test):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    
    predictions = svm.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("SVM Test accuracy "+ str(accuracy))
    
    return {"method": "SVM", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": svm}

In [8]:
from tensorflow.keras import layers
from tensorflow import keras

def classify_neuralnet(X_train, X_test, y_train, y_test, n_vars, n_classes, depth=1, nodes=10, epochs=20):
    inputs = keras.Input(shape=(n_vars,), name='obs')
    x = layers.Dense(nodes, activation='relu')(inputs)
    
    if(depth>1):
        for i in range(depth-1):
            x = layers.Dense(nodes, activation='relu')(x)
            
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='Dataset')

    display(model.summary())


    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=64,
                        epochs=epochs,
                        validation_split=0.2)

    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    print(predictions)

    accuracy = accuracy_score(y_test, predictions)
    print("Neural Network Test accuracy "+ str(accuracy))
    
    return {"method": "Net "+"-".join([str(nodes) for i in range(depth)])+ " E"+str(epochs), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": model}

In [9]:
from datetime import datetime
import pandas as pd
import time


np.random.seed(12345)

timelabel =  datetime.now().strftime("%H-%M-%S - %d-%m-%Y")


configs = [{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 500,
    "max_mu": 2,
    "max_sigma": 2,
    "max_skew": 1
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.05, 0.05, 0.05, 0.85],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 50,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.125 for x in range(8)],
    "n_vars": 5,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.5, 0.5],
    "n_vars": 2,
    "n": 1000000,
    "max_mu": 1,
    "max_sigma": 5,
    "max_skew": 5
}]



columns = ['method', 'accuracy','predictions', "model", "config"] + list(configs[0].keys())
results = pd.DataFrame(columns=columns)
results.style.format({
    'accuracy': '{:,.3%}'.format
})


#run analysis
for i, c in enumerate(configs):
    X_train, X_test, y_train, y_test = simulate_data(c["classes"], c["n_vars"], c["n"], c["max_mu"], c["max_sigma"], c["max_skew"])
    
    c_n = i+1
    
    lda = classify_lda(X_train, X_test, y_train, y_test, c["classes"], False)                                     
    results = results.append({**lda, **c, "config":c_n},ignore_index=True)
    
    qda = classify_qda(X_train, X_test, y_train, y_test, c["classes"])                                     
    results = results.append({**qda, **c, "config":c_n},ignore_index=True)
    
    logit = classify_logit(X_train, X_test, y_train, y_test)                                     
    results = results.append({**logit, **c, "config":c_n},ignore_index=True)
    
    for k in [5,10,50,100]:
        knn = classify_knn(X_train, X_test, y_train, y_test, k)
        results = results.append({**knn, **c, "config":c_n},ignore_index=True)

    bayes = classify_naivebayes(X_train, X_test, y_train, y_test, c["classes"])
    results = results.append({**bayes, **c, "config":c_n},ignore_index=True)
    
    svm = classify_svm(X_train, X_test, y_train, y_test)
    results = results.append({**svm, **c, "config":c_n},ignore_index=True)
    
    for n in [{"d":1,"n":len(c["classes"]), "e":25}, {"d":4,"n":30, "e":10}, {"d":4,"n":50, "e":50}]:
        neuralnet = classify_neuralnet(X_train, X_test, y_train, y_test, c["n_vars"], len(c["classes"]),  depth=n["d"], nodes=n["n"], epochs=n["e"])                                 
        results = results.append({**neuralnet, **c, "config":c_n},ignore_index=True)
    
    print("Results after config "+str(c_n)+" of "+str(len(configs)))

    results.sort_values(by='accuracy', ascending=False, inplace=True)
        
    display(results.style.format({
    'accuracy': '{:,.3%}'.format
    }))
    

    #saving results to file
    results.drop(columns=['model']).to_pickle("./results/config "+str(i+1)+" of "+str(len(configs))+" "+timelabel+".pkl")

LDA Test accuracy 0.5575757575757576
[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 2. 1. 1. 0. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2.
 1. 3. 3. 2. 0. 3. 3. 1. 2. 3. 1. 1. 3. 1. 3. 1. 3. 0. 2. 2. 0. 1. 3. 2.
 3. 1. 1. 3. 0. 3. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 2. 0. 3. 1.
 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 2. 0. 2. 3. 1. 3. 3. 0.
 0. 3. 2. 3. 0. 0. 0. 3. 1. 2. 0. 0. 1. 2. 1. 3. 3. 0. 0. 1. 3. 1. 3. 1.
 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 2. 3. 1. 1. 0. 3. 1. 1. 1. 0. 2.
 1. 0. 2. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 2. 2. 1.]
QDA Test accuracy 0.5272727272727272
Logistic Test accuracy 0.5636363636363636
KNN-5 Test accuracy 0.4484848484848485
KNN-10 Test accuracy 0.4909090909090909
KNN-50 Test accuracy 0.5393939393939394
KNN-100 Test accuracy 0.4727272727272727
Naive Bayes Test accuracy 0.5333333333333333
SVM Test accuracy 0.5636363636363636


C:\Users\Tim\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Model: "Dataset"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
obs (InputLayer)             [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


None

Train on 268 samples, validate on 67 samples
Epoch 1/25
268/268 [==============================] - 1s 5ms/sample - loss: 1.5093 - accuracy: 0.2127 - val_loss: 1.4274 - val_accuracy: 0.2090
Epoch 2/25
268/268 [==============================] - 0s 170us/sample - loss: 1.4943 - accuracy: 0.2090 - val_loss: 1.4182 - val_accuracy: 0.2239
Epoch 3/25
268/268 [==============================] - 0s 168us/sample - loss: 1.4851 - accuracy: 0.2201 - val_loss: 1.4117 - val_accuracy: 0.2090
Epoch 4/25
268/268 [==============================] - 0s 164us/sample - loss: 1.4772 - accuracy: 0.2201 - val_loss: 1.4051 - val_accuracy: 0.2090
Epoch 5/25
268/268 [==============================] - 0s 187us/sample - loss: 1.4696 - accuracy: 0.2276 - val_loss: 1.3992 - val_accuracy: 0.2090
Epoch 6/25
268/268 [==============================] - 0s 172us/sample - loss: 1.4628 - accuracy: 0.2276 - val_loss: 1.3934 - val_accuracy: 0.2090
Epoch 7/25
268/268 [==============================] - 0s 199us/sample - loss: 1.4

...

None

Train on 536000 samples, validate on 134000 samples
Epoch 1/10
536000/536000 [==============================] - 38s 70us/sample - loss: 0.6929 - accuracy: 0.5104 - val_loss: 0.6928 - val_accuracy: 0.5124
Epoch 2/10
536000/536000 [==============================] - 35s 66us/sample - loss: 0.6927 - accuracy: 0.5110 - val_loss: 0.6928 - val_accuracy: 0.5131
Epoch 3/10
536000/536000 [==============================] - 37s 68us/sample - loss: 0.6927 - accuracy: 0.5119 - val_loss: 0.6929 - val_accuracy: 0.5111 - accu - ETA: 0s - los
Epoch 4/10
536000/536000 [==============================] - 36s 68us/sample - loss: 0.6927 - accuracy: 0.5112 - val_loss: 0.6927 - val_accuracy: 0.5132
Epoch 5/10
536000/536000 [==============================] - 35s 65us/sample - loss: 0.6927 - accuracy: 0.5117 - val_loss: 0.6927 - val_accuracy: 0.5132
Epoch 6/10
536000/536000 [==============================] - 36s 67us/sample - loss: 0.6927 - accuracy: 0.5119 - val_loss: 0.6927 - val_accuracy: 0.5124
Epoch 7/10
53

None

Train on 536000 samples, validate on 134000 samples
Epoch 1/50
536000/536000 [==============================] - 36s 68us/sample - loss: 0.6930 - accuracy: 0.5095 - val_loss: 0.6928 - val_accuracy: 0.5107
Epoch 2/50
536000/536000 [==============================] - 37s 68us/sample - loss: 0.6928 - accuracy: 0.5109 - val_loss: 0.6928 - val_accuracy: 0.5081
Epoch 3/50
536000/536000 [==============================] - 36s 67us/sample - loss: 0.6927 - accuracy: 0.5116 - val_loss: 0.6928 - val_accuracy: 0.5121
Epoch 4/50
536000/536000 [==============================] - 37s 69us/sample - loss: 0.6927 - accuracy: 0.5114 - val_loss: 0.6929 - val_accuracy: 0.5099
Epoch 5/50
536000/536000 [==============================] - 37s 69us/sample - loss: 0.6927 - accuracy: 0.5118 - val_loss: 0.6928 - val_accuracy: 0.5096
Epoch 6/50
536000/536000 [==============================] - 37s 68us/sample - loss: 0.6927 - accuracy: 0.5120 - val_loss: 0.6928 - val_accuracy: 0.5117
Epoch 7/50
536000/536000 [==========

,method,accuracy,predictions,model,config,classes,n_vars,n,max_mu,max_sigma,max_skew
0,Net 50-50-50-50 E50,100.000%,[2 3 1 ... 1 1 3],,4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
1,Net 4 E25,100.000%,[2 3 1 ... 1 1 3],,4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
2,KNN-100,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=100, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
3,KNN-50,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=50, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
4,SVM,100.000%,[2. 3. 1. ... 1. 1. 3.],"LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, loss='squared_hinge', max_iter=1000, multi_class='ovr', penalty='l2', random_state=None, tol=0.0001, verbose=0)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
5,Naive Bayes,100.000%,[2. 3. 1. ... 1. 1. 3.],"GaussianNB(priors=[0.25, 0.25, 0.25, 0.25], var_smoothing=1e-09)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
6,Logit,100.000%,[2. 3. 1. ... 1. 1. 3.],"LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='multinomial', n_jobs=None, penalty='l2', random_state=0, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
7,Net 30-30-30-30 E10,100.000%,[2 3 1 ... 1 1 3],,4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
8,LDA,100.000%,[2. 3. 1. ... 1. 1. 3.],"LinearDiscriminantAnalysis(n_components=None, priors=[0.25, 0.25, 0.25, 0.25], shrinkage=None, solver='svd', store_covariance=False, tol=0.0001)",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5
9,KNN-10,100.000%,[2. 3. 1. ... 1. 1. 3.],"KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean', metric_params=None, n_jobs=None, n_neighbors=10, p=2, weights='uniform')",4,"[0.25, 0.25, 0.25, 0.25]",50,10000,5,5,5


Example to load old results

In [19]:
import pandas as pd

def bold(data):
    attr = 'font-weight: {}'.format("700")
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    is_max = data == data.max()
    return [attr if v else "" for v in is_max]
    
def boldlatex(data):
    #remove % and cast to float
    return list(map(lambda x: r"\\textbf{"+x+"}" if x==data.max() else x, data))
   
table = results.copy()
table['accuracy'] = pd.Series(["{0:.2f}%".format(val * 100) for val in table['accuracy']], index = table.index)
table.loc[table.method == "Net 2 E25", "method"] = "Net N_C E25"
table.loc[table.method == "Net 4 E25", "method"] = "Net N_C E25"
table.loc[table.method == "Net 8 E25", "method"] = "Net N_C E25"

table = table.pivot(index='method', columns='config', values='accuracy').sort_values(by='method')
display(table.style.apply(bold))
latex = table.apply(boldlatex).to_latex()
print(latex)

config,1,2,3,4,5,6,7,8,9,10,11
method,,,,,,,,,,,
KNN-10,49.09%,94.88%,99.24%,100.00%,32.97%,33.52%,26.92%,46.25%,85.38%,64.16%,50.23%
KNN-100,47.27%,92.36%,98.64%,100.00%,36.79%,36.70%,27.71%,48.78%,86.85%,67.30%,50.41%
KNN-5,44.85%,95.12%,99.36%,100.00%,31.73%,31.74%,26.20%,43.55%,84.65%,63.05%,50.14%
KNN-50,53.94%,93.45%,98.88%,100.00%,37.09%,36.38%,27.50%,48.87%,86.69%,66.88%,50.42%
LDA,55.76%,96.82%,99.97%,100.00%,37.42%,39.34%,28.54%,52.72%,84.45%,67.52%,51.34%
Logit,56.36%,96.82%,99.88%,100.00%,37.91%,39.55%,28.41%,52.91%,84.76%,67.49%,51.34%
Naive Bayes,53.33%,96.85%,99.97%,100.00%,36.94%,39.46%,28.46%,52.75%,84.46%,67.52%,51.37%
Net 30-30-30-30 E10,51.52%,96.42%,99.73%,100.00%,38.39%,41.33%,28.46%,54.06%,86.71%,67.60%,51.22%
Net 50-50-50-50 E50,52.73%,96.58%,99.82%,100.00%,37.45%,40.87%,28.45%,55.14%,86.47%,67.62%,51.30%


\begin{tabular}{llllllllllll}
\toprule
config &                1  &                2  &                3  &                 4  &                5  &                6  &                7  &                8  &                9  &                10 &                11 \\
method              &                   &                   &                   &                    &                   &                   &                   &                   &                   &                   &                   \\
\midrule
KNN-10              &            49.09\% &            94.88\% &            99.24\% &  \textbackslash \textbackslash textbf\{100.00\%\} &            32.97\% &            33.52\% &            26.92\% &            46.25\% &            85.38\% &            64.16\% &            50.23\% \\
KNN-100             &            47.27\% &            92.36\% &            98.64\% &  \textbackslash \textbackslash textbf\{100.00\%\} &            36.79\% &            36.70\% &            27

In [32]:
datasets = pd.DataFrame.from_dict(configs)
datasets = datasets[['max_mu', 'max_sigma', 'max_skew', 'n_vars', 'n', 'classes',]]
datasets.insert(0, "dataset", datasets.index+1)
datasets = datasets.transpose()
display(datasets)

print(datasets.to_latex())

,0,1,2,3,4,5,6,7,8,9,10
dataset,1,2,3,4,5,6,7,8,9,10,11
max_mu,2,5,5,5,1,1,1,1,1,1,1
max_sigma,2,5,5,5,10,10,10,10,10,5,5
max_skew,1,5,5,5,10,10,10,10,10,5,5
n_vars,2,5,5,50,5,5,2,5,2,2,2
n,500,10000,10000,10000,10000,100000,100000,100000,100000,100000,1000000
classes,"[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.05, 0.05, 0.05, 0.85]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.25, 0.25, 0.25, 0.25]","[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.1...","[0.5, 0.5]","[0.5, 0.5]","[0.5, 0.5]"


\begin{tabular}{llllllllllll}
\toprule
{} &                         0 &                         1 &                         2 &                         3 &                         4 &                         5 &                         6 &                                                  7 &           8 &           9 &          10 \\
\midrule
dataset   &                         1 &                         2 &                         3 &                         4 &                         5 &                         6 &                         7 &                                                  8 &           9 &          10 &          11 \\
max\_mu    &                         2 &                         5 &                         5 &                         5 &                         1 &                         1 &                         1 &                                                  1 &           1 &           1 &           1 \\
max\_sigma &                         2 &      